In [1]:
import pandas as pd
import numpy as np

from pyspark import SparkContext
from pyspark.sql import HiveContext
# sc = SparkContext()
sqlContext = HiveContext(sc)

sqlContext.sql("SET spark.sql.parquet.binaryAsString=true")
sqlContext.sql("SET spark.sql.parquet.compression.codec=snappy")

sqlContext.sql('use x5_ru_analysis')

import pandas as pd
pd.set_option('display.max_rows', None)
from cycler import cycler
import numpy as np
import datetime, time
from pyspark.sql.types import DoubleType

In [2]:
base = sqlContext.createDataFrame(pd.read_excel("Actual baskets 18012018.xlsx"))

In [4]:
base.show(5)

+--------------------+---------+-------------+-----------+--------------------+-----------------+--------------------+--------+
|    TRANSACTION_CODE|BASKET ID|MACRO_MISSION|     BANNER|        PRODUCT_NAME|       NEED_STATE|  PROD_HIER_L30_CODE|ITEM_QTY|
+--------------------+---------+-------------+-----------+--------------------+-----------------+--------------------+--------+
|20160828000916204...|      411|  CONSUME_NOW|Perekrestok|Напиток SPRITE га...|          IMPULSE|  Juice, Water, Beer|     1.0|
|20160828003618102...|      154|  CONSUME_NOW|    Karusel|TWIX Батон.ЭКСТРА...|          IMPULSE|Confectionery (Food)|     1.0|
|20160828003618102...|      154|  CONSUME_NOW|    Karusel|Напиток FLASH ENE...|          IMPULSE|  Juice, Water, Beer|     1.0|
|20160828003618102...|      154|  CONSUME_NOW|    Karusel|БОЛ.КРУЖКА Йогурт...|YOGHURTANDCURDSML|      Dairy Products|     1.0|
|20160828005239214...|      758|  CONSUME_NOW|Perekrestok|СТЕПАНОВНА Семена...|  HEALTHYSNACKING|    Gro

In [5]:
base.registerTempTable("base")

base_1 = sqlContext.sql("""
SELECT A.*,CASE WHEN NEED_STATE = 'SMOKING' THEN 'S' ELSE NULL END NS_SMOK
FROM BASE A
""")

base_1.show(5)

+--------------------+---------+-------------+-----------+--------------------+-----------------+--------------------+--------+-------+
|    TRANSACTION_CODE|BASKET ID|MACRO_MISSION|     BANNER|        PRODUCT_NAME|       NEED_STATE|  PROD_HIER_L30_CODE|ITEM_QTY|NS_SMOK|
+--------------------+---------+-------------+-----------+--------------------+-----------------+--------------------+--------+-------+
|20160828000916204...|      411|  CONSUME_NOW|Perekrestok|Напиток SPRITE га...|          IMPULSE|  Juice, Water, Beer|     1.0|   null|
|20160828003618102...|      154|  CONSUME_NOW|    Karusel|TWIX Батон.ЭКСТРА...|          IMPULSE|Confectionery (Food)|     1.0|   null|
|20160828003618102...|      154|  CONSUME_NOW|    Karusel|Напиток FLASH ENE...|          IMPULSE|  Juice, Water, Beer|     1.0|   null|
|20160828003618102...|      154|  CONSUME_NOW|    Karusel|БОЛ.КРУЖКА Йогурт...|YOGHURTANDCURDSML|      Dairy Products|     1.0|   null|
|20160828005239214...|      758|  CONSUME_NOW|Pe

In [8]:
import ast
import configparser
import logging
import pyspark.sql.functions as F
import sys
from numpy import array
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
from pyspark.sql.functions import concat
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.window import Window

basket_summ = base_1.groupBy("transaction_code")\
                     .agg(F.sum("item_qty").cast(DecimalType(16,4)).alias("item_qty"),
                         F.countDistinct("need_state").alias("cnt_needstate"),
                         F.countDistinct("ns_smok").alias("cnt_smk"))
    

In [13]:
agg_all = base_1.join(basket_summ,"transaction_code").withColumn('cnt_ns', g(agg_all.cnt_needstate, agg_all.cnt_smk)) 

agg_all.show(100)

+--------------------+---------+-------------+-----------+--------------------+--------------------+--------------------+--------+-------+--------+-------------+-------+------+
|    transaction_code|BASKET ID|MACRO_MISSION|     BANNER|        PRODUCT_NAME|          NEED_STATE|  PROD_HIER_L30_CODE|ITEM_QTY|NS_SMOK|item_qty|cnt_needstate|cnt_smk|cnt_ns|
+--------------------+---------+-------------+-----------+--------------------+--------------------+--------------------+--------+-------+--------+-------------+-------+------+
|20160828034926103...|      530|       TOP_UP|    Karusel|ХЛ.ДОМ Бат.МОС.на...|        MILKANDBREAD|Bread, Baked Good...|     1.0|   null|  2.0000|            2|      0|     2|
|20160828034926103...|      530|       TOP_UP|    Karusel|ПИСКАР.Молоко пас...|    DAIRYPRODUCTSLRG|      Dairy Products|     1.0|   null|  2.0000|            2|      0|     2|
|20160828042436103...|      762|       TOP_UP|    Karusel|ГМ Картофель ранн...|        FRESHVEGGIES|  Fruit - Veget

In [12]:
from pyspark.sql.functions import udf
def subtr(x,y):
    if x == 1:
        return x
    else:
        return x - y
    
g = udf(subtr)

agg_all1 = agg_all.withColumn('cnt_ns', g(agg_all.cnt_needstate, agg_all.cnt_smk)) 

agg_all1.show(100)

+--------------------+---------+-------------+-----------+--------------------+--------------------+--------------------+--------+-------+--------+-------------+-------+------+
|    transaction_code|BASKET ID|MACRO_MISSION|     BANNER|        PRODUCT_NAME|          NEED_STATE|  PROD_HIER_L30_CODE|ITEM_QTY|NS_SMOK|item_qty|cnt_needstate|cnt_smk|cnt_ns|
+--------------------+---------+-------------+-----------+--------------------+--------------------+--------------------+--------+-------+--------+-------------+-------+------+
|20160828034926103...|      530|       TOP_UP|    Karusel|ХЛ.ДОМ Бат.МОС.на...|        MILKANDBREAD|Bread, Baked Good...|     1.0|   null|  2.0000|            2|      0|     2|
|20160828034926103...|      530|       TOP_UP|    Karusel|ПИСКАР.Молоко пас...|    DAIRYPRODUCTSLRG|      Dairy Products|     1.0|   null|  2.0000|            2|      0|     2|
|20160828042436103...|      762|       TOP_UP|    Karusel|ГМ Картофель ранн...|        FRESHVEGGIES|  Fruit - Veget

In [5]:
trans1 = sqlContext.sql("""
SELECT A.*,B.BMAP
FROM bask_ns_distribution_ovr A
INNER JOIN
  (SELECT B.*
   FROM
     ( SELECT DISTINCT BASKET_ID 

AS TRANSACTION_CODE,
                       1 AS BMAP
      FROM x5_ru_analysis.kg_sm_pyt_52weeks
      UNION ALL SELECT 

DISTINCT BASKET_ID AS TRANSACTION_CODE,
                                2 AS BMAP
      FROM 

x5_ru_analysis.kg_sm_karusel_100per2_52weeks
      UNION ALL SELECT DISTINCT BASKET_ID AS TRANSACTION_CODE,
                 

               3 AS BMAP
      FROM x5_ru_analysis.kg_sm_perekrestok_20per_52weeks)B)B ON A.TRANSACTION_CODE = B.TRANSACTION_CODE
""")

In [6]:
trans1.count()

200767800

In [3]:
trans.count()

200767800

In [3]:
trans.show(5)

+--------------------+------------+------+------------+
|    transaction_code|basket_spend|num_ns|basket_prods|
+--------------------+------------+------+------------+
|20160828085045414...|     20200.0|     3|           4|
|20160828090050445...|     39700.0|     4|           6|
|20160828095931108...|    107400.0|     7|          11|
|20160828102040101...|    112210.0|     8|          12|
|20160828102445101...|     40710.0|     7|           8|
+--------------------+------------+------+------------+
only showing top 5 rows



In [8]:
trans.groupBy("num_ns")\
    .sum("basket_spend")\
    .show(100)

+------+-----------------+
|num_ns|sum(basket_spend)|
+------+-----------------+
|    31|     1.78546109E8|
|    32|      7.4881071E7|
|    33|      4.6986453E7|
|    34|        3549527.0|
|    35|        1845210.0|
|    36|        8135726.0|
|     1|  7.8499802274E11|
|     2| 9.17752693112E11|
|     3|1.019220673446E12|
|     4|1.066561588633E12|
|     5|1.080823780718E12|
|     6|1.060901096233E12|
|     7|1.004856659185E12|
|     8| 9.19387333666E11|
|     9| 8.15988714414E11|
|    10| 7.06413107671E11|
|    11| 6.00964416261E11|
|    12| 5.03726209023E11|
|    13| 4.18259332706E11|
|    14| 3.44604123946E11|
|    15| 2.81154270647E11|
|    16| 2.26974563528E11|
|    17| 1.81038237535E11|
|    18| 1.41949390933E11|
|    19|  1.0939083104E11|
|    20|  8.2239749985E10|
|    21|  5.9192177214E10|
|    22|   4.170401242E10|
|    23|  2.8078512626E10|
|    24|   1.827421525E10|
|    25|  1.1360895654E10|
|    26|    6.612346977E9|
|    27|    3.756036017E9|
|    28|    1.928392442E9|
|

In [7]:
trans1.groupBy("num_ns","bmap")\
    .count()\
    .show(300)

+------+----+--------+
|num_ns|bmap|   count|
+------+----+--------+
|     4|   1| 7002589|
|    31|   2|     117|
|     4|   2| 7713928|
|    31|   3|       2|
|     4|   3| 6238074|
|    15|   1|  105404|
|    15|   2|  637775|
|    15|   3|  126493|
|    26|   1|      41|
|    26|   2|    6784|
|    26|   3|     272|
|    10|   1| 1027664|
|    10|   2| 2355753|
|    10|   3|  979233|
|    21|   1|    3046|
|    21|   2|   86562|
|    21|   3|    6692|
|     5|   1| 5343149|
|    32|   2|      50|
|     5|   2| 6424457|
|     5|   3| 4706981|
|    16|   1|   62921|
|    16|   2|  477684|
|    16|   3|   81249|
|    27|   1|      15|
|    27|   2|    3612|
|    27|   3|     109|
|    11|   1|  678973|
|    11|   2| 1850234|
|    11|   3|  669958|
|    22|   1|    1466|
|    22|   2|   57016|
|    22|   3|    3684|
|     6|   1| 4053901|
|    33|   2|      25|
|     6|   2| 5381726|
|     6|   3| 3571748|
|    17|   1|   36726|
|    17|   2|  353644|
|    17|   3|   51213|
|    28|   

In [4]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from pyspark_dist_explore import hist

fig, ax = plt.subplots()
hist(ax, trans['num_ns'], bins = 20, color=['red'])

ImportError: No module named pyspark_dist_explore

In [37]:
sc.version

u'1.6.0'

In [6]:
import pandas as pd
import numpy as np

from pyspark import SparkContext
from pyspark.sql import HiveContext
# sc = SparkContext()
sqlContext = HiveContext(sc)

sqlContext.sql("SET spark.sql.parquet.binaryAsString=true")
sqlContext.sql("SET spark.sql.parquet.compression.codec=snappy")

sqlContext.sql('use x5_ru_analysis')

import pandas as pd
pd.set_option('display.max_rows', None)
from cycler import cycler
import numpy as np
import datetime, time
from pyspark.sql.types import DoubleType

from numpy import array
from pyspark.mllib.clustering import GaussianMixture, GaussianMixtureModel


clus_data = sqlContext.table("clus_data")
##########################################################################
#------------------------------------------Standardisation----------------
##########################################################################
'''clus_data_stand = stand(df_stand=clus_data,
                        key="transaction_code",
                        feature_vars=clus_data.columns[1:])
clus_data_stand.write.mode("overwrite").parquet("clus_data_stand")'''
clus_data_stand = sqlContext.read.parquet("clus_data_stand")
##########################################################################
#--------------------------------------------Optimal K--------------------
##########################################################################
# making elbow curve to find optimal k for the data
# function to plot the elbow curve and seeds

from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

#df = spark.read.option("inferSchema", "true").option("header", "true").csv("whole_customers_data.csv")
cols = clus_data_stand.columns[1:]

vectorAss = VectorAssembler(inputCols=cols, outputCol="assembled_features")
vdf = vectorAss.transform(clus_data_stand)

featuresrdd = clus_data_stand.select(*cols).rdd.map(list)
model = GaussianMixture.train(featuresrdd, 10, convergenceTol=0.0001,maxIterations=100, seed=10)

#a = "transaction_code"
clusterrdd = vdf.map(lambda x: (x["transaction_code"], model.predict(x['assembled_features'])))


In [33]:
model = GaussianMixture.train(featuresrdd, 10, convergenceTol=0.00001,maxIterations=100, seed=10)


In [18]:
vdf

DataFrame[transaction_code: string, ns_Activity_s: double, ns_Alcohol_s: double, ns_Automotive_s: double, ns_Baby_care_and_Children_s: double, ns_Baby_food_s: double, ns_Bakery_s: double, ns_Beer_and_Crisps_s: double, ns_Bread_and_butter_s: double, ns_Breakfast_Milk_and_eggs_s: double, ns_Breakfast_Yoghurt_and_bread_s: double, ns_Breakfast_milk_and_cereal_s: double, ns_Candies_s: double, ns_Cheese_and_bread_s: double, ns_Childrens_goods_s: double, ns_Clothing_and_accessories_s: double, ns_Confectionery_s: double, ns_Convenience_cooking_s: double, ns_Cooking_s: double, ns_Cooled_meat_s: double, ns_Cream_and_cheese_s: double, ns_Dairy_products_s: double, ns_Deli_s: double, ns_Desserts_s: double, ns_Eat_as_you_go_s: double, ns_Electronics_s: double, ns_Festival_event_s: double, ns_Festivity_preparation_s: double, ns_Fish_cooking_s: double, ns_Fish_snacking_s: double, ns_Fresh_meat_and_fish_s: double, ns_Frozen_s: double, ns_Frozen_Desserts_s: double, ns_Fruits_and_Vegetables_s: double, ns

In [21]:
clusterrdd = vdf.map(lambda x: (x['transaction_code'], model.predict(x['assembled_features'])))
clusterrdd

Exception: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

In [34]:
clusters = model.predictSoft(featuresrdd)

In [35]:
print(clusters.take(1000))

[array('d', [0.10000000000000003, 0.10000000000000003, 0.10000000000000003, 0.10000000000000003, 0.10000000000000003, 0.10000000000000003, 0.10000000000000003, 0.10000000000000003, 0.10000000000000003, 0.10000000000000003]), array('d', [0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002]), array('d', [0.09999999999999999, 0.09999999999999999, 0.09999999999999999, 0.09999999999999999, 0.09999999999999999, 0.09999999999999999, 0.09999999999999999, 0.09999999999999999, 0.09999999999999999, 0.09999999999999999]), array('d', [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]), array('d', [0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002, 0.10000000000000002]), array('d', [0.10000000000000003, 0.1000

In [23]:
df = sqlContext.createDataFrame(clusters)

ValueError: The first row in RDD is empty, can not infer schema

In [22]:
cluster_data_gmm = clus_data.join(clusters,
                              "transaction_code",
                              "left_outer")

AttributeError: 'PipelinedRDD' object has no attribute '_jdf'

In [ ]:
cluster_exp_1 =  clusterrdd.toDF(['transaction_code', 'cluster'])

#cluster_exp_1 = model.predict(featuresrdd).collect()

cluster_data_gmm = clus_data.join(cluster_exp_1,
                              "transaction_code",
                              "left_outer")

sqlContext.sql("drop table if exists cluster_data_GMM")
cluster_data_gmm.write.saveAsTable("cluster_data_GMM",mode='overwrite')

In [2]:
# dsn_1.write.mode("overwrite").parquet("dsn_1.parquet")

dsn_1 = sqlContext.read.parquet("dsn_1.parquet")

dsn_1.show()

+--------------------+-------------------+------------------+-----------+------------------+
|          need_state|       p_5_ns_spend|     p_95_ns_spend|p_5_ns_prod|      p_95_ns_prod|
+--------------------+-------------------+------------------+-----------+------------------+
|              Frozen|  4179.847828911388|   67788.673661297|        1.0|  2.61072384604254|
|                  GM|-3607.8343826021073|11487.268518518522|        1.0|               1.0|
|              Bakery| 1183.7047632164001|24491.158440688767|        1.0| 2.616182619843523|
|    Healthy_snacking| 2005.2032522365716|31459.282408888877|        1.0|2.7469949557086606|
|Home_decor_and_St...|  1549.859372036183|50661.578911564604|        1.0|  3.92515076542446|
|      Festival_event|  2977.559852884151| 93794.17031481482|        1.0| 3.010577342047928|
|       Eat_as_you_go|  2657.493807848186|34893.177599999995|        1.0|2.9440387279407334|
|           Baby_food|  2162.868527757518| 99860.43939049583|        1

In [3]:
need_state_data = sqlContext.read.parquet("need_state_data.parquet")

need_state_data.write.saveAsTable("need_state_data",
                                       mode='overwrite')
bask_need_states_spend

In [4]:
bask_need_states_spend = sqlContext.read.parquet("bask_need_states_spend.parquet")
bask_need_states_spend.write.saveAsTable("bask_need_states_spend",mode='overwrite')

bask_need_states1 = sqlContext.read.parquet("bask_need_states1.parquet")
bask_need_states1.write.saveAsTable("bask_need_states1",mode='overwrite')

bask_need_states2 = sqlContext.read.parquet("bask_need_states2.parquet")
bask_need_states2.write.saveAsTable("bask_need_states2",mode='overwrite')

bask_need_states3 = sqlContext.read.parquet("bask_need_states3.parquet")
bask_need_states3.write.saveAsTable("bask_need_states3",mode='overwrite')

In [2]:
baskets1 = sqlContext.sql(""" 
SELECT BASKET_ID AS TRANSACTION_CODE,
       NS_HARM AS NEED_STATE,
       ITEM_SPEND AS NET_SPEND_AMT,
       CATEGORY_DESC_ENG AS PROD_HIER_L30_CODE,
       A.PRODUCT_CODE AS CATEGORY_NAME
FROM x5_ru_analysis.kg_sm_pyt_52weeks A
INNER JOIN
  (SELECT A.*,
          B.NS_HARM
   FROM
     (SELECT A.*,
             CONCAT('_',GROUP_CODE,'_',DUNN_CAT_ENGLISH) AS NEW_CAT
      FROM x5_ru_analysis.kg_prod_sm_new2 A) A
   INNER JOIN X5_RU_ANALYSIS.AK_NS_MAP_PYT1 B ON A.NEW_CAT = B.KARU_CATEGORY_FINAL) B ON A.PRODUCT_CODE = B.PRODUCT_CODE
WHERE item_spend > 0
  AND item_qty > 0
""")

baskets2 = sqlContext.sql(""" 
SELECT BASKET_ID AS TRANSACTION_CODE,
       NS_HARM AS NEED_STATE,
       ITEM_SPEND AS NET_SPEND_AMT,
       CATEGORY_DESC_ENG AS PROD_HIER_L30_CODE,
       A.PRODUCT_CODE AS CATEGORY_NAME
FROM x5_ru_analysis.kg_sm_karusel_100per2_52weeks A
INNER JOIN (SELECT A.*,B.NS_HARM FROM (SELECT A.*,CONCAT('_',GROUP_CODE,'_',DUNN_CAT_ENGLISH) AS NEW_CAT 
FROM x5_ru_analysis.ak_prod_karusel_2 A) A INNER JOIN 
X5_RU_ANALYSIS.AK_NS_MAP_KARUSEL1 B ON A.NEW_CAT = B.KARU_CATEGORY_FINAL ) B 
ON A.PRODUCT_CODE = B.PRODUCT_CODE
""")

baskets3 = sqlContext.sql(""" 
SELECT BASKET_ID AS TRANSACTION_CODE,
       NS_HARM AS NEED_STATE,
       ITEM_SPEND AS NET_SPEND_AMT,
       CATEGORY_DESC_ENG AS PROD_HIER_L30_CODE,
       A.PRODUCT_CODE AS CATEGORY_NAME
FROM x5_ru_analysis.kg_sm_perekrestok_20per_52weeks A
INNER JOIN (SELECT A.*,B.NS_HARM FROM (SELECT A.*,CONCAT('_',GROUP_CODE,'_',DUNN_CAT_ENGLISH) AS NEW_CAT 
FROM x5_ru_analysis.ak_prod_perekrestok_1 A) A INNER JOIN 
X5_RU_ANALYSIS.AK_NS_MAP_PTOK1 B ON A.NEW_CAT = B.KARU_CATEGORY_FINAL ) B 
ON A.PRODUCT_CODE = B.PRODUCT_CODE
""")

df_concat = baskets1.unionAll(baskets2)
df_final = df_concat.unionAll(baskets3)

print df_final.count()



1259660311


In [4]:
df_final.registerTempTable("df_final")

t2 = sqlContext.sql(""" 
SELECT DISTINCT TRANSACTION_CODE FROM 
df_final
""")

print t2.count()

201407054


In [ ]:
b1 = sqlContext.sql(""" 
SELECT DISTINCT BASKET_ID AS TRANSACTION_CODE,1 AS BMAP FROM 
x5_ru_analysis.kg_sm_pyt_52weeks
""")

b2 = sqlContext.sql(""" 
SELECT DISTINCT BASKET_ID AS TRANSACTION_CODE,2 AS BMAP FROM 
x5_ru_analysis.kg_sm_karusel_100per2_52weeks
""")

b3 = sqlContext.sql(""" 
SELECT DISTINCT BASKET_ID AS TRANSACTION_CODE,3 AS BMAP FROM 
x5_ru_analysis.kg_sm_perekrestok_20per_52weeks
""")

b4 = b1.unionAll(b2)
b5 = b4.unionAll(b3)


percent_back = 0.05

# use this if you want an (almost) exact number of samples
# sample_count = 200
# percent_back = sample_count / posts.count()

frac = dict(
    (e.BMAP, percent_back) 
    for e 
    in b5.select('BMAP').distinct().collect()
)

b6 = b5.sampleBy('BMAP', frac, 1234)

df_final.registerTempTable("df_final")
b6.registerTempTable("b6")

trans = sqlContext.sql(""" 
SELECT A.*
FROM df_final A 
INNER JOIN b6 B 
on A.TRANSACTION_CODE = B.TRANSACTION_CODE
""")

trans.registerTempTable("trans")

t1 = sqlContext.sql(""" 
SELECT DISTINCT TRANSACTION_CODE FROM 
TRANS
""")
# trans.write.saveAsTable(database+".need_states_dist",
#                                    mode='overwrite') 
print t1.count()

In [6]:
print t1.count()

10070203


In [2]:
core_prods = pd.read_csv("ns_map_ovr_split.csv")

core_prods_sdf = sqlContext.createDataFrame(core_prods)

core_prods_sdf.registerTempTable("core_prods_sql")
sqlContext.sql("drop table if exists AK_ns_map_ovr_split")
sqlContext.sql("create table AK_ns_map_ovr_split as select * from core_prods_sql");

In [2]:
import ast
import configparser
import logging
import matplotlib
matplotlib.use('PDF')
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from time import time
import sys
from numpy import array
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.feature import StandardScaler
from pyspark.sql.functions import concat
import pandas as pd


# Standardisation Function# Standardisation Function
def stand(df_stand, key=None, feature_vars=None):
    """
        Function Name: stand(df_stand,key=None,feature_vars=None)
        Use: Standardisation
        Input:
        df_stand: this is the pyspark dataframe to be standardised
        Key: Unique row identifier
        Feature_vars: column names to be standardised
        Output: a pyspark dataframe containing the standardised result
    """
    def extract(row):
        return (row[key], ) + tuple(float(x) for x in row._2.values)
    df1_stand = df_stand.select(key, *feature_vars)
    features_stand = df1_stand.map(lambda row: row[1:])
    #     Standardizing the data
    standardizer_stand = StandardScaler(True, True)
    model_stand = standardizer_stand.fit(features_stand)
    features_transform_stand = model_stand.transform(features_stand)
    lab_stand = df1_stand.map(lambda row: row[0])
    transformedData_stand = lab_stand.zip(features_transform_stand)
    DfVec_stand = sqlContext.createDataFrame(transformedData_stand)
    df2_stand = DfVec_stand.\
        withColumnRenamed('_1', key)\
        .rdd.map(extract).toDF([key] + feature_vars)
    df2_stand = df2_stand.join(df_stand
                               .select([c for c in df_stand.columns
                                        if c not in feature_vars]), key)

    return df2_stand

# Class for clustering


class KMeansCluster():
    """ Class for K Means clustering """

    def __init__(self, data_frame, features, observation,
                 initialization_mode="k-means||",
                 initialization_steps=5,
                 epsilon=0.0001):

        self.data_frame = data_frame
        self.initialization_mode = initialization_mode
        self.initialization_steps = initialization_steps
        self.epsilon = epsilon

        if set(features).issubset(set(data_frame.columns)):
            self.features = features
            self.featuresrdd = data_frame.select(*features).rdd.map(list)
        else:
            raise Exception("""Some feature columns are
                            not in the data 
                            """)

        if {observation}.issubset(set(data_frame.columns)):
            self.observation = observation
        else:
            raise Exception("""observation column
                            not in the data 
                            """)

        self.v_df = VectorAssembler(
            inputCols=self.features,
            outputCol='assembled_features').transform(self.data_frame)

        self._centres = None
        self._k = None
        self._model = None
        self._tssse = None
        self._n = None

    def get_cluster(self, k=None, centres=None, max_iteration=100, seed=None):
        """ Will output clusters for given k or centres """
        if centres is not None:
            self._centres = centres
            self._model = KMeansModel(self._centres)
        elif k is not None:
            self.train(k, max_iteration=max_iteration)
        if self._model is None:
            raise Exception("""Model does not exist, 
            please use KMeansCluster.train before using predict""")
        else:
            a = self.observation
            b = self._model
            clusterrdd = self.v_df\
                             .map(lambda x: (x[a], b.predict(x['assembled_features'])))
            return clusterrdd.toDF([a, 'cluster'])

    def get_centres(self):
        """ Will output centres for given kmeans object """
        if self._model is None:
            print """model does not exist, 
            use KMeansCluster.train to define the model"""
            return None
        else:
            self._k = len(self._centres)
            return self._centres, self._k

    def optimal_clusters(self, statistic='r_sq', start=2, end=5):
        """ Will output stat value for clusters for iterations from start to end """
        optimal_centres = {}
        optimal_statistic = {}
        for i in range(start, end + 1):
            self.train(k=i)
            if statistic == 'r_sq':
                stat = self.r_sq()
            elif statistic == 'pseudo_f':
                stat = self.pseudo_f()
            elif statistic == 'wssse':
                stat = self.wssse()
            else:
                raise Exception("statistic not defined")

            optimal_centres[i] = self._centres
            optimal_statistic[i] = stat

        return optimal_statistic, optimal_centres

    def train(self, k, max_iteration=100, seed=None):
        """ Will train for the given k """
        self._k = k
        print """Creating {} clusters""".format(k)

        trained_model = KMeans.train(self.featuresrdd, self._k,
                                     maxIterations=max_iteration,
                                     epsilon=self.epsilon,
                                     initializationMode=self.initialization_mode,
                                     initializationSteps=self.initialization_steps,
                                     seed=seed)

        self._model = trained_model
        self._centres = self._model.clusterCenters
        return trained_model

    def predict(self, measure):
        """ Will predict clusters for given k or centres """
        if len(measure) != len(self.features):
            raise Exception("""number of items in measure does
                               not match the number of features""")

        if self._model is None:
            print "Model does not exist, please use KMeansCluster.\
                    train before using predict"
            return None
        else:
            return self._model.predict(measure)

    def wssse(self):
        """ Will output wsse stat for the given model """
        if self._model is None:
            print "Model does not exist, \
                   please train a model using KMeansCluster.train"
            return None
        else:
            return self._model.computeCost(self.featuresrdd)

    def r_sq(self):
        """ Will output RSQ stat for the given model """
        if self._model is None:
            print "Model does not exist,\
                   please train a model using KMeansCluster.train"
            return None
        else:
            if self._tssse is None:
                temp_clus = KMeans.train(self.featuresrdd, 1)
                self._tssse = temp_clus.computeCost(self.featuresrdd)
            rsq = 1 - (self.wssse() / self._tssse)
        return rsq

    def pseudo_f(self):
        """ Will output pseudo f stat for the given model """

        if self._model is None:
            print "Model does not exist, \
                   please train a model using KMeansCluster.train"
            return None
        else:
            if self._n is None:
                self._n = self.data_frame.count()
            rsq = self.r_sq()
            pseudof = (rsq / (1 - rsq)) * ((self._n - self._k) / (self._k - 1))
        return pseudof


def elbow(
        data_frame, features, observation,
        stat, start, end, name):
    """ This function gives the elbow statistics and plots for the same """

    kmeans = KMeansCluster(data_frame=data_frame,
                           features=features,
                           observation=observation)
    stat = stat
    start = start
    end = end
    elbow, centres = kmeans.optimal_clusters(statistic=stat,
                                             start=start,
                                             end=end)

    # plotting elbow curve
    plt.figure(figsize=(10, 8))
    plt.title('Elbow Curve First Stage', fontsize=20)
    plt.xlabel('Number of Cluster', fontsize=15)
    plt.ylabel(stat + ' value', fontsize=15)
    plt.xticks(elbow.keys())
    plt.plot(elbow.keys(), elbow.values())
    plt.savefig(name + ".pdf")
    return elbow, centres, kmeans


# clus_data = sqlContext.table(database+".clus_data")
# stage = int(app_config["cluster"]["stage"])
# if stage == 1:
#     ##########################################################################
#     #------------------------------------------Standardisation----------------
#     ##########################################################################
#     clus_data_stand = stand(df_stand=clus_data,
#                             key="transaction_code",
#                             feature_vars=clus_data.columns[1:])
#     clus_data_stand.write.mode("overwrite").parquet("clus_data_stand")
#     clus_data_stand = sqlContext.read.parquet("clus_data_stand")
#     ##########################################################################
#     #--------------------------------------------Optimal K--------------------
#     ##########################################################################
#     # making elbow curve to find optimal k for the data
#     # function to plot the elbow curve and seeds
#     stat = str(app_config["cluster"]["stat"])
#     start = int(app_config["cluster"]["start"])
#     end = int(app_config["cluster"]["end"])
#     elbow, centres, kmeans = elbow(data_frame=clus_data_stand,
#                                    features=clus_data_stand.columns[1:],
#                                    observation="transaction_code",
#                                    stat=stat, start=start, end=end,
#                                    name="elbow_test1")
#     print("Look at elbow_test1.pdf for the elbow curve")
#     print("Stage 1 completed")
# ##########################################################################
# #----------------------------------------Clustering First Stage-----------
# ##########################################################################
# start_time = time()
# if stage == 2:
#     clus_data_stand = sqlContext.read.parquet("clus_data_stand")
#     # scoring the data
#     k_first = int(app_config["cluster"]["k_first"])
#     seed_used = str(app_config["cluster"]["seed_used"])
#     stat = str(app_config["cluster"]["stat"])
#     kmeans = KMeansCluster(data_frame=clus_data_stand,
#                            features=clus_data_stand.columns[1:],
#                            observation="transaction_code")
#     elbow, centres = kmeans.optimal_clusters(statistic=stat,
#                                              start=k_first,
#                                              end=k_first)
#     seed_first = pd.DataFrame(centres[centres.keys()[0]])
#     seed_first.columns = clus_data_stand.columns[1:]
#     seed_first["num_clus"] = centres.keys()[0]
#     seed_first.set_index("num_clus").to_csv("centres_first")
#     if seed_used == "yes":
#         seeds = centres[k_first]
#         cluster_exp_1 = kmeans.get_cluster(centres=seeds)
#     if seed_used == "no" or seed_used == "":
#         cluster_exp_1 = kmeans.get_cluster(k=k_first)
#     cluster_data = clus_data.join(cluster_exp_1,
#                                   "transaction_code",
#                                   "left_outer")\
#         .na.drop()
#     sqlContext.sql("drop table if exists "+database+".cluster_data1")
#     cluster_data.write.saveAsTable(database+".cluster_data1",
#                                    mode='overwrite')
#     cluster_data = sqlContext.table(database+".cluster_data1")
#     cluster_data.groupBy("cluster").agg(F.countDistinct("transaction_code")
#                                         .alias("baskets")).show()
#     print("Decide which cluster/clusters should be further broken down")
    
#     print("Stage 2 completed in", end_time)

# ##########################################################################
# #----------------------------------------Clustering Second Stage----------
# ##########################################################################
# if stage == 3:
#     big_clus = str(app_config["cluster"]["big_clus"])
#     cluster_data = sqlContext.table(database+".cluster_data1")\
#                              .na.drop()

#     if big_clus == "auto":
#         clus_size = cluster_data.groupBy("cluster")\
#             .agg(F.countDistinct("transaction_code")
#                  .alias("baskets"))
#         big_clus_size = clus_size.agg(
#             getattr(F, "max")("baskets").alias("baskets"))
#         big_clus_num = clus_size.join(big_clus_size,
#                                       "baskets",
#                                       "inner")\
#             .select("cluster")
#         big_clus_data = cluster_data.join(big_clus_num,
#                                           "cluster",
#                                           "inner")\
#             .drop("cluster")
#         clus_break_1 = stand(df_stand=big_clus_data,
#                              key="transaction_code",
#                              feature_vars=big_clus_data.columns[1:])
#         sqlContext.sql("drop table if exists "+database+".clus_break_1")
#         clus_break_1.write.saveAsTable(database+".clus_break_1",
#                                        mode='overwrite')
#         clus_break_1 = sqlContext.table(database+".clus_break_1")
#         stat = str(app_config["cluster"]["stat_big"])
#         start = int(app_config["cluster"]["start_big"])
#         end = int(app_config["cluster"]["end_big"])
#         elbow_big, centres_big, kmeans_big = elbow(data_frame=clus_break_1,
#                                                    features=clus_break_1.columns[1:],
#                                                    observation="transaction_code",
#                                                    stat=stat, start=start,
#                                                    end=end,
#                                                    name="elbow_test2" + str(big_clus))

#     elif big_clus == "none":
#         logger.info("No second stage clustering performed")
#     else:
#         big_data = cluster_data[getattr(F, "col")("cluster") == big_clus]
#         big_clus_data = big_data.where(
#             getattr(F, "col")("cluster") == int(big_clus))
#         clus_break_1 = stand(df_stand=big_clus_data,
#                              key="transaction_code",
#                              feature_vars=big_clus_data.columns[1:])
#         sqlContext.sql("drop table if exists "+database+".clus_break_1")
#         clus_break_1.write.saveAsTable(database+".clus_break_1",
#                                        mode='overwrite')
#         clus_break_1 = sqlContext.table(database+".clus_break_1")
#         stat = str(app_config["cluster"]["stat_big"])
#         start = int(str(app_config["cluster"]["start_big"]))
#         end = int(str(app_config["cluster"]["end_big"]))
#         elbow_big, centres_big, kmeans_big = elbow(data_frame=clus_break_1,
#                                                    features=clus_break_1.columns[1:],
#                                                    observation="transaction_code",
#                                                    stat=stat, start=start,
#                                                    end=end,
#                                                    name="elbow_test2_" + str(big_clus))
#         print("Look at elbow_test2 " + str(big_clus) + " \
#                 to decide the number of cluster")
#     print("Stage 3 completed")

# if stage == 4:
#     big_clus = str(app_config["cluster"]["big_clus"])
#     cluster_data = sqlContext.table(database+".cluster_data1")
#     clus_break_1 = sqlContext.table(database+".clus_break_1")
#     # clustering the data
#     if big_clus == "auto":
#         clus_size = cluster_data.groupBy("cluster")\
#             .agg(F.countDistinct("transaction_code")
#                  .alias("baskets"))
#         big_clus_size = clus_size.agg(
#             getattr(F, "max")("baskets").alias("baskets"))
#         big_clus_num = clus_size.join(big_clus_size,
#                                       "baskets",
#                                       "inner").select("cluster")
#         k_second = int(app_config["cluster"]["k_second"])
#         seed_used_big = str(app_config["cluster"]["seed_used_big"])
#         stat = str(app_config["cluster"]["stat_big"])
#         start = k_second
#         end = k_second
#         kmeans_big = KMeansCluster(data_frame=clus_break_1,
#                                    features=clus_break_1.columns[1:],
#                                    observation="transaction_code")
#         elbow_big, centres_big = kmeans_big.optimal_clusters(statistic=stat,
#                                                              start=start,
#                                                              end=end)
#         seed_second = pd.DataFrame(centres_big[centres_big.keys()[0]])
#         seed_second.columns = clus_break_1.columns[1:]
#         seed_second["num_clus"] = centres_big.keys()[0]
#         seed_second.set_index("num_clus").to_csv("centres_second")
#         if seed_used_big == "yes":
#             seeds = centres_big[k_second]
#             cluster_exp_11 = kmeans_big.get_cluster(centres=seeds)
#         if seed_used_big == "no" or seed_used_big == "":
#             cluster_exp_11 = kmeans_big.get_cluster(k=k_second)
#         clus_num = big_clus_num.collect()[0][0]
#         cluster_exp_11 = cluster_exp_11.withColumn("cluster_2",
#                                                    concat(getattr(F, "lit")(clus_num),
#                                                           getattr(F, "lit")(
#                                                               "."),
#                                                           getattr(F, "col")("cluster")))\
#             .drop("cluster")
#         cluster_data1 = cluster_data[~getattr(F, "col")("cluster")
#                                      .isin([clus_num])]\
#             .withColumn("cluster_2", getattr(F, "col")("cluster"))
#         cluster_data2 = cluster_data.join(cluster_exp_11,
#                                           "transaction_code",
#                                           "inner")
#         clus_data_fin = cluster_data1.unionAll(cluster_data2)
#         fin_missions = clus_data_fin.select("transaction_code",
#                                             "cluster",
#                                             "cluster_2")
#         sqlContext.sql("drop table if exists "+database+".Missions")
#         fin_missions.write.saveAsTable(database+".Missions",mode='overwrite')
        
#         logger.info("Missions Extracted")
#     elif big_clus == "none":
#         fin_missions = cluster_data.select("transaction_code", "cluster")
#         sqlContext.sql("drop table if exists "+database+".Missions")
#         fin_missions.write.saveAsTable(database+".Missions",mode='overwrite')
        
#         logger.info("Missions Extracted")
#     else:
#         cluster_data_fin = cluster_data[~getattr(F, "col")("cluster").isin([big_clus])]\
#             .withColumn("cluster_2",
#                         getattr(F, "col")("cluster"))\
#             .select("transaction_code",
#                     "cluster",
#                     "cluster_2")
#         k_second = int(app_config["cluster"]["k_second"])
#         seed_used_big = str(app_config["cluster"]["seed_used_big"])
#         stat = str(app_config["cluster"]["stat_big"])
#         start = k_second
#         end = k_second
#         kmeans_big = KMeansCluster(data_frame=clus_break_1,
#                                    features=clus_break_1.columns[1:],
#                                    observation="transaction_code")
#         elbow_big, centres_big = kmeans_big.optimal_clusters(statistic=stat,
#                                                              start=start,
#                                                              end=end)
#         seed_second = pd.DataFrame(centres_big[centres_big.keys()[0]])
#         seed_second.columns = clus_break_1.columns[1:]
#         seed_second["num_clus"] = centres_big.keys()[0]
#         seed_second.set_index("num_clus").to_csv("centres_second")
#         if seed_used_big == "yes":
#             seeds = centres_big[k_second]
#             cluster_exp_11 = kmeans_big.get_cluster(centres=seeds)
#         if seed_used_big == "no" or seed_used_big == "":
#             cluster_exp_11 = kmeans_big.get_cluster(k=k_second)
#         clus_num = int(big_clus)
#         cluster_exp_11 = cluster_exp_11.withColumn("cluster_2",
#                                                    concat(getattr(F, "lit")(str(big_clus)),
#                                                           getattr(F, "lit")(
#                                                               "."),
#                                                           getattr(F, "col")("cluster")))\
#             .drop("cluster")
#         cluster_data2 = cluster_data.join(cluster_exp_11,
#                                           "transaction_code",
#                                           "inner")\
#             .select("transaction_code",
#                     "cluster",
#                     "cluster_2")
#         cluster_data_fin = cluster_data_fin.unionAll(cluster_data2)
#         fin_missions = cluster_data_fin.select(
#             "transaction_code", "cluster", "cluster_2")
#         sqlContext.sql("drop table if exists "+database+".Missions")
#         fin_missions.write.saveAsTable(database+".Missions",
#                                        mode='overwrite')
#         print("Missions Extracted")
#     print("Stage 4 completed")

# # Setup if the job is run using spark-submit
# if environment is True:
#     sc.stop()


In [16]:
import time
startTime = time.time()

baskets1 = sqlContext.sql(""" 
SELECT BASKET_ID,
       NS_HARM,
       SUM(ITEM_SPEND) AS SPEND
FROM x5_ru_analysis.kg_sm_pyt_52weeks A
INNER JOIN
  (SELECT A.*,
          B.NS_HARM
   FROM
     (SELECT A.*,
             CONCAT('_',GROUP_CODE,'_',DUNN_CAT_ENGLISH) AS NEW_CAT
      FROM x5_ru_analysis.kg_prod_sm_new2 A) A
   INNER JOIN X5_RU_ANALYSIS.AK_NS_MAP_PYT B ON A.NEW_CAT = B.KARU_CATEGORY_FINAL) B ON A.PRODUCT_CODE = B.PRODUCT_CODE
WHERE item_spend > 0
  AND item_qty > 0
GROUP BY BASKET_ID,
         NS_HARM 
""")

baskets2 = sqlContext.sql(""" 
SELECT BASKET_ID,NS_HARM,SUM(ITEM_SPEND) AS SPEND
FROM x5_ru_analysis.kg_sm_karusel_100per2_52weeks A
INNER JOIN (SELECT A.*,B.NS_HARM FROM (SELECT A.*,CONCAT('_',GROUP_CODE,'_',DUNN_CAT_ENGLISH) AS NEW_CAT 
FROM x5_ru_analysis.ak_prod_karusel_2 A) A INNER JOIN 
X5_RU_ANALYSIS.AK_NS_MAP_KARUSEL B ON A.NEW_CAT = B.KARU_CATEGORY_FINAL ) B 
ON A.PRODUCT_CODE = B.PRODUCT_CODE
GROUP BY BASKET_ID,NS_HARM
""")

baskets3 = sqlContext.sql(""" 
SELECT BASKET_ID,NS_HARM,SUM(ITEM_SPEND) AS SPEND
FROM x5_ru_analysis.kg_sm_perekrestok_20per_52weeks A
INNER JOIN (SELECT A.*,B.NS_HARM FROM (SELECT A.*,CONCAT('_',GROUP_CODE,'_',DUNN_CAT_ENGLISH) AS NEW_CAT 
FROM x5_ru_analysis.ak_prod_perekrestok_1 A) A INNER JOIN 
X5_RU_ANALYSIS.AK_NS_MAP_PTOK B ON A.NEW_CAT = B.KARU_CATEGORY_FINAL ) B 
ON A.PRODUCT_CODE = B.PRODUCT_CODE
GROUP BY BASKET_ID,NS_HARM
""")

df_concat = baskets1.unionAll(baskets2)
baskets = df_concat.unionAll(baskets3)


# baskets_sample = baskets.sample(False, 0.01, 1234)

# baskets_sample1.registerTempTable("baskets_sample1")

# sqlContext.sql("""drop table if exists ak_trans_pyt_smp510""")
# trans_sample1 = sqlContext.sql("""create table ak_trans_pyt_smp510 as
#                             Select DISTINCT BASKET_ID,
#                             CONCAT('_',GROUP_CODE,'_',DUNN_CAT_ENGLISH) AS NEW_CAT,
#                             1 AS VALUE
#                             From x5_ru_analysis.kg_sm_pyt_52weeks A
#                             inner join x5_ru_analysis.kg_prod_sm_new2 b
#                             on a.product_code = b.product_code                            
#                             """)

# elapsedTime = time.time() - startTime
# print "Time taken for Pyt is :"+str(elapsedTime)

In [17]:
baskets.show(10)

+--------------------+----------------+-------+
|           BASKET_ID|         NS_HARM|  SPEND|
+--------------------+----------------+-------+
|20161216175252682...|Impulse purchase| 6390.0|
|20170402212047Q17...|Impulse purchase| 6990.0|
|20170816195454Q33...|Impulse purchase| 8700.0|
|20161014184113552...|Impulse purchase| 3386.0|
|20170420105557870...|Impulse purchase| 6830.0|
|20170727111352861...|Impulse purchase| 3590.0|
|20161106134433549...|Impulse purchase| 3594.0|
|20170119145510561...|Impulse purchase| 8730.0|
|20170315133706539...|Impulse purchase| 5195.0|
|20170525124904522...|Impulse purchase|11075.0|
+--------------------+----------------+-------+
only showing top 10 rows



In [20]:
baskets.columns

['BASKET_ID', 'NS_HARM', 'SPEND']

In [23]:
baskets_pivot = baskets_sample.groupBy("BASKET_ID").pivot("NS_HARM").sum("SPEND")

Py4JJavaError: An error occurred while calling o853.pivot.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:57)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:526)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:234)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:381)
py4j.Gateway.invoke(Gateway.java:214)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:79)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:68)
py4j.GatewayConnection.run(GatewayConnection.java:209)
java.lang.Thread.run(Thread.java:748)

The

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 43932)
----------------------------------------


Traceback (most recent call last):
  File "/usr/lib64/python2.7/SocketServer.py", line 295, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python2.7/SocketServer.py", line 649, in __init__
    self.handle()
  File "/opt/cloudera/parcels/CDH/lib/spark/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/opt/cloudera/parcels/CDH/lib/spark/python/pyspark/serializers.py", line 545, in read_int
    raise EOFError
EOFError


In [26]:
baskets_pivot.show(10)

+--------------------+--------+-------+----------------------+---------------+----------------+-------+------------------------+-------------+-------------------+-------+-----------+--------------+--------+--------------+---------------------+-------------+----+-----------+------------+---------------+----------------+--------+---------+----------------+-----------+-----------+-------------+--------+------+------+-----+-------+----------------+-------+--------------+---------------+
|           BASKET_ID|Activity|Alcohol|Baby_care_and_Children|Beer_and_Crisps|Bread_and_butter|Candies|Clothing_and_accessories|Confectionery|Convenience_cooking|Cooking|Cooled_meat|Dairy_products|Desserts|Festival_event|Festivity_preparation|Fish_snacking|  GM|Garden_care|Ham_&_Cheese|Healthy_Impulse|Healthy_snacking|Homeware|Household|Impulse_purchase|Kitchenware|Milk_top_up|Personal_care|Pet_Care|Picnic| Salad|Shoes|Smoking|Social_gathering|Staples|Tea_and_Coffee|Wine_and_Cognac|
+--------------------+--

In [38]:
baskets_pivot.columns[1:]

['Activity',
 'Alcohol',
 'Baby_care_and_Children',
 'Beer_and_Crisps',
 'Bread_and_butter',
 'Candies',
 'Clothing_and_accessories',
 'Confectionery',
 'Convenience_cooking',
 'Cooking',
 'Cooled_meat',
 'Dairy_products',
 'Desserts',
 'Festival_event',
 'Festivity_preparation',
 'Fish_snacking',
 'GM',
 'Garden_care',
 'Ham_&_Cheese',
 'Healthy_Impulse',
 'Healthy_snacking',
 'Homeware',
 'Household',
 'Impulse_purchase',
 'Kitchenware',
 'Milk_top_up',
 'Personal_care',
 'Pet_Care',
 'Picnic',
 'Salad',
 'Shoes',
 'Smoking',
 'Social_gathering',
 'Staples',
 'Tea_and_Coffee',
 'Wine_and_Cognac']

In [5]:
baskets_pivot.fillna(0)

DataFrame[BASKET_ID: string, Activity: double, Alcohol: double, Baby_care_and_Children: double, Beer_and_Crisps: double, Bread_and_butter: double, Candies: double, Clothing_and_accessories: double, Confectionery: double, Convenience_cooking: double, Cooking: double, Cooled_meat: double, Dairy_products: double, Desserts: double, Festival_event: double, Festivity_preparation: double, Fish_snacking: double, GM: double, Garden_care: double, Ham_&_Cheese: double, Healthy_Impulse: double, Healthy_snacking: double, Homeware: double, Household: double, Impulse_purchase: double, Kitchenware: double, Milk_top_up: double, Personal_care: double, Pet_Care: double, Picnic: double, Salad: double, Shoes: double, Smoking: double, Social_gathering: double, Staples: double, Tea_and_Coffee: double, Wine_and_Cognac: double]

In [10]:
##########################################################################
#------------------------------------------Standardisation----------------
##########################################################################
clus_data_stand = stand(df_stand=baskets_pivot,
                        key="basket_id",
                        feature_vars=baskets_pivot.columns[1:])
# clus_data_stand.write.mode("overwrite").parquet("clus_data_stand")
# clus_data_stand = sqlContext.read.parquet("clus_data_stand")
##########################################################################
#--------------------------------------------Optimal K--------------------
##########################################################################
# making elbow curve to find optimal k for the data
# function to plot the elbow curve and seeds
# stat = str(app_config["cluster"]["stat"])
# start = int(app_config["cluster"]["start"])
# end = int(app_config["cluster"]["end"])



In [ ]:
clus_data_stand.columns[1:]

In [11]:
elbow, centres, kmeans = elbow(data_frame=clus_data_stand,
                               features=clus_data_stand.columns[1:],
                               observation="basket_id",
                               stat='r_sq', start=2, end=20,
                               name="elbow_test1")
print("Look at elbow_test1.pdf for the elbow curve")
print("Stage 1 completed")

Creating 2 clusters


IllegalArgumentException: u'requirement failed'